Name: Robert (Bobby) Fisher
CWID: 11187333

In [1]:
# import required modules
from pyspark.sql.functions import udf
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import HashingTF
from pyspark.sql.functions import udf
from pyspark.sql.functions import when
from pyspark.sql.types import StringType
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
import re

In [3]:
# load sms data into a DataFrame
sms_df = spark.read.format("csv") \
    .option("header", "true") \
    .load('hdfs://localhost:9000/user/pistolpete/spark_data/sms.data')

In [4]:
# examine schema and some rows of data
sms_df.printSchema()
sms_df.show()


root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [5]:
# function to remove non-alpha-numeric characters
def removeNonAlphaNum(x):
    y = re.sub(r'[^\w\s]', '', x)
    z = y.lower()
    return z

# create a user-defined function and use through Spark SQL
removePunctuationUDF = udf(removeNonAlphaNum, StringType()) 
sms_df_1 = sms_df.select(
    when(sms_df["class"] == "spam", 1).otherwise(0).alias("class"), 
    removePunctuationUDF(sms_df['text']).alias("text"))
	
# examine new schema
sms_df_1.printSchema()

root
 |-- class: integer (nullable = false)
 |-- text: string (nullable = true)



In [6]:
# Create Feature transformer objects 
tk_obj = Tokenizer(inputCol = "text", outputCol = "tokens")
swr_obj = StopWordsRemover(inputCol = "tokens", outputCol = "filtered")
htf_obj = HashingTF(numFeatures = 25, inputCol = "filtered", outputCol = "features")

# Transform training data
sms_df_2 = tk_obj.transform(sms_df_1)
sms_df_3 = swr_obj.transform(sms_df_2)
sms_df_4 = htf_obj.transform(sms_df_3)

# Split DataFrame into training and testing parts
df_train, df_test = sms_df_4.randomSplit([0.7, 0.3], seed = 42)
print('Train: ', df_train.count())
print('Test : ', df_test.count())

Train:  3939
Test :  1635


In [7]:
# Create random forest classifier object
rf_obj = RandomForestClassifier(
    labelCol = "class", 
    featuresCol = "features", 
    numTrees = 50)

# Fit the model using the training data
rf_model = rf_obj.fit(df_train)

# Score testing data
pred_rf = rf_model.transform(df_test)
pred_rf_1 = pred_rf.select(
    when(pred_rf["class"] == 1, "spam").otherwise("ham").alias("class"), 
    when(pred_rf["prediction"] == 1, "spam").otherwise("ham").alias("prediction"))

In [8]:
# Function to compute Accuracy
def Accuracy(df, orig_col = "class", pred_col = "prediction"):
    n = df.count()
    n_match = df.filter(df[orig_col] == df[pred_col]).count()
    return n_match * 100.0 / n

# Compute and print accuracy and confusion matrix
rf_acc_1 = Accuracy(pred_rf_1)
print("Accuracy : ", rf_acc_1)
print("Confusion Matrix: ")
pred_rf_1.crosstab("class", "prediction").show()

Accuracy :  87.82874617737004
Confusion Matrix: 
+----------------+----+----+
|class_prediction| ham|spam|
+----------------+----+----+
|            spam| 194|  31|
|             ham|1405|   5|
+----------------+----+----+



In [9]:
# Extract param map and update a couple of values
rf_param_map = rf_obj.extractParamMap()
rf_param_map.update(
	{rf_obj.numTrees:100, 
	rf_obj.impurity:'entropy'})

# Fit new model based on param map and score
rf_model_2 = rf_obj.fit(df_train, rf_param_map)
pred_rf = rf_model_2.transform(df_test)
pred_rf_2 = pred_rf.select(
    when(pred_rf["class"] == 1, "spam").otherwise("ham").alias("class"), 
    when(pred_rf["prediction"] == 1, "spam").otherwise("ham").alias("prediction"))

In [10]:
# Compute and print accuracy and confusion matrix
rf_acc_2 = Accuracy(pred_rf_2)
print("Accuracy : ", rf_acc_2)
print("Confusion Matrix: ")
pred_rf_2.crosstab("class", "prediction").show()

Accuracy :  87.5229357798165
Confusion Matrix: 
+----------------+----+----+
|class_prediction| ham|spam|
+----------------+----+----+
|            spam| 199|  26|
|             ham|1405|   5|
+----------------+----+----+

